In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.8MB/s 
     |████████████████████████████████| 901kB 46.8MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 8.1MB/s 


In [11]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
src_text = [
    """ 
Agent: Hi. Thanks for calling. Net service. How can I help you? 
Customer: Hi. Yes, my Internet keeps disconnecting. 
Agent: Sorry To hear about that, Sir. I would definitely help you with this issue, but before that, may I know your nine digit customer ID? Oh.
Customer: Come on. Can you just search by my registration phone number? 
Agent: Sorry. Sir. Our system only recognizes nine digit customer ID.
Customer:  Oh. That's ridiculous. Let me check. Okay, I found it. It's 345-93-9420. 
Agent: Great.Thanks, Mr. Daniel. Let me transfer you to our internet Department as they are the only ones who can help you with the issue.
Agent: Hi Thanks for calling. How may I help? You?
Customer: Already told you guys. Just now my Internet keeps disconnecting. 
Agent: Very Sorry to hear that Sir. Can you please confirm if there's uninterrupted power supply at your house?
Customer:  Are You serious? You think I'm stupid? Of course there's no issue with my poor supplier at my home.
Agent: We are following our standard diagnostic procedure. Currently calm down and restrain from using any cuss words.
Customer: Calm down? Did you just ask me to calm down? I've been very calm till now, However, you guys absolutely useless. I'm going to end this contract.
     
 """
]
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
print(model.device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt")

batch = batch.to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)


cuda:0


In [13]:
tgt_text[0]

'This is a verbatim transcript of the phone conversation between a customer and an agent of a telecommunications company. I would definitely help you with this issue, but before that, may I know your nine digit customer ID?'

In [19]:
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# import torch
# src_text = [
#     """ 
# Agent Hi. Thanks for calling. Net service. How can I help you? 
# Customer Hi. Yes, my Internet keeps disconnecting. 
# Agent Sorry To hear about that, Sir. I would definitely help you with this issue, but before that, may I know your nine digit customer ID? Oh.
# Customer Come on. Can you just search by my registration phone number? 
# Agent Sorry. Sir. Our system only recognizes nine digit customer ID.
# Customer  Oh. That's ridiculous. Let me check. Okay, I found it. It's 345-93-9420. 
# Agent Great.Thanks, Mr. Daniel. Let me transfer you to our internet Department as they are the only ones who can help you with the issue.
# Agent Hi Thanks for calling. How may I help? You?
# Customer Already told you guys. Just now my Internet keeps disconnecting. 
# Agent Very Sorry to hear that Sir. Can you please confirm if there's uninterrupted power supply at your house?
# Customer  Are You serious? You think I'm stupid? Of course there's no issue with my poor supplier at my home.
# Agent We are following our standard diagnostic procedure. Currently calm down and restrain from using any cuss words.
# Customer Calm down? Did you just ask me to calm down? I've been very calm till now, However, you guys absolutely useless. I'm going to end this contract.
     
#  """
# ]
# model_name = 'google/pegasus-xsum'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
# print(model.device)
# batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt")

# batch = batch.to(device)
# translated = model.generate(**batch)
# tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)


cuda:0


In [20]:
# tgt_text

['The following is a verbatim transcript of the call centre conversation between the customer and the agent. I would definitely help you with this issue, but before that, may I know your nine digit customer ID?']

In [5]:
#Fine tuning

In [3]:
!pip install datasets

     |████████████████████████████████| 225kB 8.5MB/s 
     |████████████████████████████████| 245kB 11.0MB/s 
     |████████████████████████████████| 112kB 13.6MB/s 


In [6]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch


class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

      
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset


def prepare_fine_tuning(model_name, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=1000,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=200,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=50,                  # number of update steps before evaluation
      warmup_steps=50,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset             # evaluation dataset
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=1000,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=50,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=50,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
    )

  return trainer



In [5]:

if __name__=='__main__':
  # use XSum dataset as example, with first 1000 docs as training data
  from datasets import load_dataset
  dataset = load_dataset("xsum")
  train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]
  
  # use Pegasus Large model as base for fine-tuning
  model_name = 'google/pegasus-large'
  train_dataset, _, _ = prepare_data(model_name, train_texts, train_labels)
  trainer = prepare_fine_tuning(model_name, train_dataset)
  import gc 

# Your code with pytorch using GPU

  # gc.collect()  
  trainer.train()
  gc.collect()
  torch.cuda.empty_cache()

Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


Step,Training Loss


RuntimeError: ignored

In [7]:
model_name = 'google/pegasus-large'
train_dataset, _, _ = prepare_data(model_name, train_texts, train_labels)
trainer = prepare_fine_tuning(model_name, train_dataset)
import gc 

# Your code with pytorch using GPU

# gc.collect()  
trainer.train()
gc.collect()
torch.cuda.empty_cache()

RuntimeError: ignored